In [51]:
import numpy as np
import scipy.optimize as op

In [52]:
n_movies=10
n_users=5
n_features=3 # comedy,action,romance etc. applies to both movie and user

In [53]:
# movies x user rating matrix where ratings are from 1 to 10
#each column is a single users rating of all movies
# 0 value if the user has not rated the movie
ratings =np.array([
 [8, 4, 0, 0, 4],
 [0, 0, 8, 10, 4],
 [8, 10, 0, 0, 6],
 [10, 10, 8, 10, 10],
 [0, 0, 0, 0, 0],
 [2, 0, 4, 0, 6],
 [8, 6, 4, 0, 0],
 [0, 0, 6, 4, 0],
 [0, 6, 0, 4, 10],
 [0, 4, 6, 8, 8]
])

In [54]:
print ratings
print ratings.shape

[[ 8  4  0  0  4]
 [ 0  0  8 10  4]
 [ 8 10  0  0  6]
 [10 10  8 10 10]
 [ 0  0  0  0  0]
 [ 2  0  4  0  6]
 [ 8  6  4  0  0]
 [ 0  0  6  4  0]
 [ 0  6  0  4 10]
 [ 0  4  6  8  8]]
(10, 5)


In [55]:
did_rate=(ratings!=0)*1
print did_rate
print did_rate.shape

[[1 1 0 0 1]
 [0 0 1 1 1]
 [1 1 0 0 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [1 0 1 0 1]
 [1 1 1 0 0]
 [0 0 1 1 0]
 [0 1 0 1 1]
 [0 1 1 1 1]]
(10, 5)


In [56]:
movies={
1:"Harold and Kumar Escape From Guantanamo Bay (2008)",
2:"Ted (2012)",
3:"Straight Outta Compton (2015)",
4:"A Very Harold and Kumar Christmas (2011)",
5:"Notorious (2009)",
6:"Get Rich Or Die Tryin' (2005)",
7:"Frozen (2013)",
8:"Tangled (2010)",
9:"Cinderella (2015)",
10:"Toy Story 3 (2010)"
}

In [57]:
sample_rating=np.zeros((n_movies,1)) #column vector
sample_rating[0] = 7
sample_rating[4] = 8
sample_rating[7] = 3

In [58]:
ratings =np.append(sample_rating,ratings, axis=1) # axis=1 append as column. i.e first column here
did_rate = np.append(((sample_rating != 0) * 1), did_rate, axis = 1)

In [59]:
print ratings
print did_rate

[[  7.   8.   4.   0.   0.   4.]
 [  0.   0.   0.   8.  10.   4.]
 [  0.   8.  10.   0.   0.   6.]
 [  0.  10.  10.   8.  10.  10.]
 [  8.   0.   0.   0.   0.   0.]
 [  0.   2.   0.   4.   0.   6.]
 [  0.   8.   6.   4.   0.   0.]
 [  3.   0.   0.   6.   4.   0.]
 [  0.   0.   6.   0.   4.  10.]
 [  0.   0.   4.   6.   8.   8.]]
[[1 1 1 0 0 1]
 [0 0 0 1 1 1]
 [0 1 1 0 0 1]
 [0 1 1 1 1 1]
 [1 0 0 0 0 0]
 [0 1 0 1 0 1]
 [0 1 1 1 0 0]
 [1 0 0 1 1 0]
 [0 0 1 0 1 1]
 [0 0 1 1 1 1]]


In [60]:
def normalize_ratings(ratings, did_rate):
    num_movies = ratings.shape[0]
    
    ratings_mean = np.zeros(shape = (num_movies, 1))
    ratings_norm = np.zeros(shape = ratings.shape)
    
    for i in range(num_movies): 
        # Get all the indexes where there is a 1
        idx = np.where(did_rate[i] == 1)[0]
        #  Calculate mean rating of ith movie only from user's that gave a rating
        ratings_mean[i] = np.mean(ratings[i, idx])
        ratings_norm[i, idx] = ratings[i, idx] - ratings_mean[i]
    
    return ratings_norm, ratings_mean

In [61]:
ratings, ratings_mean = normalize_ratings(ratings, did_rate)

In [63]:
print ratings
print ratings_mean

[[ 1.25        2.25       -1.75        0.          0.         -1.75      ]
 [ 0.          0.          0.          0.66666667  2.66666667 -3.33333333]
 [ 0.          0.          2.          0.          0.         -2.        ]
 [ 0.          0.4         0.4        -1.6         0.4         0.4       ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.         -2.          0.          0.          0.          2.        ]
 [ 0.          2.          0.         -2.          0.          0.        ]
 [-1.33333333  0.          0.          1.66666667 -0.33333333  0.        ]
 [ 0.          0.         -0.66666667  0.         -2.66666667  3.33333333]
 [ 0.          0.         -2.5        -0.5         1.5         1.5       ]]
[[ 5.75      ]
 [ 7.33333333]
 [ 8.        ]
 [ 9.6       ]
 [ 8.        ]
 [ 4.        ]
 [ 6.        ]
 [ 4.33333333]
 [ 6.66666667]
 [ 6.5       ]]


In [64]:
n_movies=ratings.shape[0]
n_users = ratings.shape[1]
n_features = 3

In [101]:

def unroll_params(X_and_theta, num_users, num_movies, num_features):
	# Retrieve the X and theta matrixes from X_and_theta, based on their dimensions (num_features, num_movies, num_movies)
	# --------------------------------------------------------------------------------------------------------------
	# Get the first 30 (10 * 3) rows in the 48 X 1 column vector
	first_30 = X_and_theta[:num_movies * num_features]
	# Reshape this column vector into a 10 X 3 matrix
	X = first_30.reshape((num_features, num_movies)).transpose()
	# Get the rest of the 18 the numbers, after the first 30
	last_18 = X_and_theta[num_movies * num_features:]
	# Reshape this column vector into a 6 X 3 matrix
	theta = last_18.reshape(num_features, num_users ).transpose()
	return X, theta


In [102]:

def calculate_cost(X_and_theta, ratings, did_rate, num_users, num_movies, num_features, reg_param):
	X, theta = unroll_params(X_and_theta, num_users, num_movies, num_features)
	
	# we multiply (element-wise) by did_rate because we only want to consider observations for which a rating was given
	cost = np.sum( (X.dot( theta.T ) * did_rate - ratings) ** 2 ) / 2
	# '**' means an element-wise power
	regularization = (reg_param / 2) * (np.sum( theta**2 ) + np.sum(X**2))
	return cost + regularization

In [103]:

def calculate_gradient(X_and_theta, ratings, did_rate, num_users, num_movies, num_features, reg_param):
	X, theta = unroll_params(X_and_theta, num_users, num_movies, num_features)
	
	# we multiply by did_rate because we only want to consider observations for which a rating was given
	difference = X.dot( theta.T ) * did_rate - ratings
	X_grad = difference.dot( theta ) + reg_param * X
	theta_grad = difference.T.dot( X ) + reg_param * theta
	
	# wrap the gradients back into a column vector 
	return np.r_[X_grad.T.flatten(), theta_grad.T.flatten()]


In [127]:
reg_param = 0
movie_features = np.random.randn( n_movies, n_features )
user_prefs = np.random.randn( n_users, n_features )
initial_X_and_theta = np.r_[movie_features.T.flatten(), user_prefs.T.flatten()]

# perform gradient descent, find the minimum cost (sum of squared errors) and optimal values of X (movie_features) and Theta (user_prefs)

minimized_cost_and_optimal_params = op.fmin_cg(calculate_cost, fprime=calculate_gradient, x0=initial_X_and_theta,
                                               args=(ratings, did_rate, n_users, n_movies, n_features, reg_param), 
                                               maxiter=100, disp=True, full_output=True ) 


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 84
         Function evaluations: 135
         Gradient evaluations: 135


In [128]:
cost, optimal_movie_features_and_user_prefs = minimized_cost_and_optimal_params[1], minimized_cost_and_optimal_params[0]

In [129]:
movie_features, user_prefs = unroll_params(optimal_movie_features_and_user_prefs, n_users, n_movies, n_features)

In [130]:
print movie_features
print user_prefs

[[ 1.05010992 -1.09442748  0.79634644]
 [ 1.21019499 -1.76674484  1.58331589]
 [-1.41038521 -1.02044863 -0.10691135]
 [-0.47484785 -0.24142671 -1.04605059]
 [ 1.92332055 -1.27992889  0.1994818 ]
 [-0.65319348  1.18790163 -0.7190741 ]
 [-0.26573231 -1.56478309 -0.10926483]
 [ 1.05598033  0.57936887  0.01379071]
 [ 0.73681287  2.11913423 -0.03433014]
 [ 1.19731191 -0.36637113 -1.55350554]]
[[-0.76093659 -0.94478166  1.27466069]
 [ 1.06986237 -1.42212389 -0.53982226]
 [-1.62523861  0.25529682  0.29645777]
 [ 0.99048566  1.05146336  0.83726428]
 [ 0.47032574 -1.42622691 -0.26671773]
 [ 0.48190764  1.39047341 -0.92206728]]


In [131]:
all_predictions = movie_features.dot( user_prefs.T )
print all_predictions

[[  1.24999945e+00   2.24999903e+00  -1.74999996e+00   5.56120860e-01
    1.84239594e+00  -1.75000131e+00]
 [  2.76649700e+00   2.95256298e+00  -1.94851367e+00   6.66667150e-01
    2.66666648e+00  -3.33333328e+00]
 [  1.90103917e+00  -5.55673838e-07   2.00000051e+00  -2.55944373e+00
    8.20565991e-01  -2.00000264e+00]
 [ -7.43934937e-01   3.99998248e-01   3.99995759e-01  -1.60000212e+00
    3.99996345e-01   3.99998795e-01]
 [ -2.24898551e-08   3.77022103e+00  -3.39347866e+00   7.26242075e-01
    2.67685086e+00  -1.03677986e+00]
 [ -1.54184434e+00  -1.99999822e+00   1.15168768e+00   1.20504114e-06
   -1.80964118e+00   2.00000140e+00]
 [  1.54130822e+00   2.00000201e+00   1.85992649e-06  -1.99999967e+00
    2.13589788e+00  -2.20309819e+00]
 [ -1.33333268e+00   2.98374774e-01  -1.56422062e+00   1.66666499e+00
   -3.33334976e-01   1.30176604e+00]
 [ -2.60654631e+00  -2.20685088e+00  -6.66665935e-01   2.92925116e+00
   -2.66666775e+00   3.33333025e+00]
 [ -2.54513015e+00   2.64060096e+00  

In [132]:
sample_prediction = all_predictions[:, 0:1] + ratings_mean
print sample_prediction
print sample_rating
print ratings_mean

[[  6.99999945]
 [ 10.09983033]
 [  9.90103917]
 [  8.85606506]
 [  7.99999998]
 [  2.45815566]
 [  7.54130822]
 [  3.00000066]
 [  4.06012036]
 [  3.95486985]]
[[ 7.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 8.]
 [ 0.]
 [ 0.]
 [ 3.]
 [ 0.]
 [ 0.]]
[[ 5.75      ]
 [ 7.33333333]
 [ 8.        ]
 [ 9.6       ]
 [ 8.        ]
 [ 4.        ]
 [ 6.        ]
 [ 4.33333333]
 [ 6.66666667]
 [ 6.5       ]]
